In [ ]:
from fastai.metrics import accuracy
from fastai.optimizer import OptimWrapper
from fastai.vision.all import *
from PIL import Image

from torch import optim
import torch.nn as nn
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import AdamW
import cv2

import gc
import glob
import inspect
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from functools import partial

from skimage.metrics import structural_similarity as ssim
from skimage import color, transform
from sklearn.model_selection import StratifiedKFold

import warnings 
warnings.filterwarnings('ignore')
set_seed(3, reproducible=True)

In [ ]:
trn_path = '/kaggle/input/zindidata/images/images'
files    = get_image_files(trn_path) 

In [ ]:
Train            = pd.read_csv('/kaggle/input/zindidata/Train.csv')
Test             = pd.read_csv('/kaggle/input/zindidata/Test.csv')
SampleSubmission = pd.read_csv('/kaggle/input/zindidata/SampleSubmission.csv')

In [ ]:
damage = Train.damage.unique()

In [ ]:
IMG_HEIGHT = IMG_WIDTH = 224
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
INCEPTION_MEAN = INCEPTION_STD = (0.5, 0.5, 0.5)


class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        
        return aug_img


def get_train_transforms(mean_std):
    augmentations = [
        A.HorizontalFlip(p=.5),
        A.VerticalFlip(p=.5),
        A.RandomRotate90(p=.5),
        A.ShiftScaleRotate(
            shift_limit=0.2, scale_limit=0.3, 
            rotate_limit=45, border_mode=0, p=.4
        ),
        A.RandomBrightnessContrast(brightness_limit=(-0.15,0.2), contrast_limit=(-0.1, 0.1), p=0.5),
        A.Resize(IMG_HEIGHT, IMG_WIDTH),
        A.Cutout(
            max_h_size=int(IMG_HEIGHT*0.2),
            max_w_size=int(IMG_WIDTH*0.2),
            num_holes=2,
            p=.4,
        ),
    ]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)


def get_valid_transforms(mean_std):
    augmentations = [A.Resize(IMG_HEIGHT, IMG_WIDTH)]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)

def get_item_tfms(mean_std='imagenet'):
    return [AlbumentationsTransform(get_train_transforms(mean_std), get_valid_transforms(mean_std))]

In [ ]:
def train(arch, item, batch, epoch=7):
    dblock = DataBlock(blocks = (ImageBlock, CategoryBlock(vocab=damage)),
                       get_x = ColReader('filename', pref = '/kaggle/input/zindidata/images/images/'),
                       get_y = ColReader('damage'),
                       splitter = RandomSplitter(valid_pct = 0.2, seed = 3),
                       item_tfms = item, #Resize(320, method = 'squish'),
                       batch_tfms = batch, #aug_transforms(size = 128, min_scale = 0.75)
                      )

    dls = dblock.dataloaders(Train)
    learn = vision_learner(dls, arch, loss_func = nn.CrossEntropyLoss(), metrics=accuracy).to_fp16()
    learn.fine_tune(epoch, 0.018)  
    dls.rng.seed(3)
    return learn

In [ ]:
learn1 = train('convit_small.fb_in1k', get_item_tfms(mean_std='inception')
               , aug_transforms(mult=0.9, do_flip=True, flip_vert=True,
                 max_rotate=33.2, min_zoom=0.6,
                 max_zoom=2, max_lighting=0.4,
                 max_warp=0.5, p_affine=0.6, size = 224, min_scale= 0.7)
               , epoch=10)

In [ ]:
learn2 = train('convnext_small.in12k', get_item_tfms(mean_std='inception'), aug_transforms(mult=0.9, do_flip=True, flip_vert=True,
                 max_rotate=13.2, min_zoom=0.6,
                 max_zoom=1.9, max_lighting=0.4,
                 max_warp=0.4, p_affine=0.60, size = 224, min_scale= 0.7), epoch=10)

In [ ]:
learn3 = train('deit_small_distilled_patch16_224.fb_in1k', get_item_tfms(mean_std='inception'), aug_transforms(mult=0.9, do_flip=True, flip_vert=True,
                 max_rotate=13.2, min_zoom=0.6,
                 max_zoom=1.9, max_lighting=0.4,
                 max_warp=0.4, p_affine=0.52, size = 224, min_scale= 0.7), epoch=10)

In [ ]:
valid = learn1.dls.valid
preds1 , targets = learn1.tta(dl=valid)
accuracy(preds1 , targets)

In [ ]:
preds1 = F.softmax(preds1, dim=1)
preds1

In [ ]:
valid = learn2.dls.valid
preds2 , targets = learn2.tta(dl=valid)
accuracy(preds2 , targets)

In [ ]:
preds2 = F.softmax(preds2, dim=1)
preds2

In [ ]:
valid = learn3.dls.valid
preds3 , targets = learn3.tta(dl=valid)
accuracy(preds3 , targets)

In [ ]:
preds3 = F.softmax(preds3, dim=1)
preds3

In [ ]:
ensemble_preds = (preds1 + preds2 + preds3) / 3
accuracy(ensemble_preds , targets)

In [ ]:
ensemble_preds

In [ ]:
test_dl = learn1.dls.test_dl('/kaggle/input/zindidata/images/images/' + Test['filename'])
preds1 , _ = learn1.tta(dl=test_dl)

In [ ]:
preds1 = F.softmax(preds1, dim=1)

In [ ]:
submission_sf = pd.DataFrame({
    
    'ID': Test['ID'],
    'DR': preds1[:, 0].squeeze().numpy(),
    'G' : preds1[:, 1].squeeze().numpy(),
    'ND': preds1[:, 2].squeeze().numpy(),
    'WD': preds1[:, 3].squeeze().numpy(),
    'other': preds1[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('convit4.csv', index=False)

In [ ]:
submission_sf

In [ ]:
test_dl = learn2.dls.test_dl('/kaggle/input/zindidata/images/images/' + Test['filename'])
preds2 , _ = learn2.tta(dl=test_dl)

In [ ]:
preds2 = F.softmax(preds2, dim=1)

In [ ]:
submission_sf = pd.DataFrame({
    
    'ID': Test['ID'],
    'DR': preds2[:, 0].squeeze().numpy(),
    'G' : preds2[:, 1].squeeze().numpy(),
    'ND': preds2[:, 2].squeeze().numpy(),
    'WD': preds2[:, 3].squeeze().numpy(),
    'other': preds2[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('convnext4.csv', index=False)

In [ ]:
submission_sf

In [ ]:
test_dl = learn3.dls.test_dl('/kaggle/input/zindidata/images/images/' + Test['filename'])
preds3 , _ = learn3.tta(dl=test_dl)

In [ ]:
preds3 = F.softmax(preds3, dim=1)

In [ ]:
submission_sf = pd.DataFrame({
    
    'ID': Test['ID'],
    'DR': preds3[:, 0].squeeze().numpy(),
    'G' : preds3[:, 1].squeeze().numpy(),
    'ND': preds3[:, 2].squeeze().numpy(),
    'WD': preds3[:, 3].squeeze().numpy(),
    'other': preds3[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('deit_small4.csv', index=False)

In [ ]:
submission_sf

In [ ]:
ens_preds = (preds1 + preds2 + preds3) / 3

In [ ]:
submission_sf = pd.DataFrame({
    'ID': Test['ID'],
    'DR': ens_preds[:, 0].squeeze().numpy(),
    'G' : ens_preds[:, 1].squeeze().numpy(),
    'ND': ens_preds[:, 2].squeeze().numpy(),
    'WD': ens_preds[:, 3].squeeze().numpy(),
    'other': ens_preds[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('ens4.csv', index=False)

In [ ]:
submission_sf